# Tutorial: causal inference methods made easy for applied resarchers/epidemiologists/statisticians 

### ICON-LSHTM, LONDON, 16th October 2020

Miguel Angel Luque Fernandez, PhD

Assistant Professor of Epidemiology and Biostatistics

Matthew Smith, PhD

Research Fellow

Paul Zivich

University of North Carolina at Chapel Hill

Inequalities in Cancer Outcomes Network, LSHTM, London, UK

Copyright (c) 2020 Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NON-INFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Bug reports: miguel-angel.luque at lshtm.ac.uk	

The rhc dataset can be dowloaded at http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets

In [1]:
# Importing libraries for the tutorial
import patsy
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

print("patsy      ", patsy.__version__)
print("numpy      ", np.__version__)
print("pandas     ", pd.__version__)
print("statsmodels", sm.__version__)

patsy       0.5.1
numpy       1.19.1
pandas      1.1.0
statsmodels 0.11.1


## Setting up the Data

In [7]:
# Box 1: Setting up the data
data = pd.read_csv("rhc.csv")
data.rename(columns={"rhc": "A", 
                     "death_d30": "Y",
                     "gender": "C",
                     "age": "W1",
                     "edu": "W2",
                     "race": "W3",
                     "carcinoma": "W4",
            }, inplace=True)
data['A'] = np.where(data['A'] == "Yes", 1, 0)
data['C'] = np.where(data['C'] == "Female", 1, 0)

data = data[["Y", "A", "C", "W1", "W2", "W3", "W4"]].copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Y       5735 non-null   int64  
 1   A       5735 non-null   int64  
 2   C       5735 non-null   int64  
 3   W1      5735 non-null   float64
 4   W2      5735 non-null   float64
 5   W3      5735 non-null   object 
 6   W4      5735 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 313.8+ KB


## Naive estimate of the ATE

In [13]:
# Box 2: Regression naive approach
fm = smf.ols("Y ~ A + C", data).fit()
print(np.round(fm.params['A'], 5))  # ATE = 0.07352
fm.summary()  # Full model results

0.07352


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     16.59
Date:                Fri, 04 Dec 2020   Prob (F-statistic):           6.58e-08
Time:                        13:57:32   Log-Likelihood:                -3812.9
No. Observations:                5735   AIC:                             7632.
Df Residuals:                    5732   BIC:                             7652.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3077      0.010     31.454      0.000       0.288       0.327
A              0.0735      0.013      5.739      0.000       0.048       0.099
C             -0.0027      0.013     -0.219      0.826      -0.027       0.022
==============================================================================
Omnibus:                    36985.427   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              993.118
Skew:                           0.696   Prob(JB):                    2.22e-216
Kurtosis:                       1.511   Cond. No.                         2.93
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
# Box 3: Marginal probabilities
pr_c1 = np.mean(data['C'])
pr_c0 = 1 - pr_c1
print("Prop. Female", np.round(pr_c1, 2))
print("Prop. Male  ", np.round(pr_c0, 2))

Prop. Female 0.44
Prop. Male   0.56


## 3. G-Formula

### 3.1 Non-parametric g-formula

In [41]:
# Box 4: Non-parametric g-formula for the ATE
pr_y_a1c1 = np.mean(data.loc[(data['C'] == 1) & (data['A'] == 1), 'Y'])
pr_y_a0c1 = np.mean(data.loc[(data['C'] == 1) & (data['A'] == 0), 'Y'])
pr_y_a1c0 = np.mean(data.loc[(data['C'] == 0) & (data['A'] == 1), 'Y'])
pr_y_a0c0 = np.mean(data.loc[(data['C'] == 0) & (data['A'] == 0), 'Y'])

ate = (pr_y_a1c1 - pr_y_a0c1)*pr_c1 + (pr_y_a1c0 - pr_y_a0c0)*pr_c0
print("ATE", np.round(ate, 6))

ATE 0.073692


In [42]:
# Box 5: Non-parametric g-formula for the ATT
pr_c1_a1 = np.mean(data.loc[data['A'] == 1, 'C'])
pr_c0_a1 = 1 - pr_c1_a1

att = (pr_y_a1c1 - pr_y_a0c1)*pr_c1_a1 + (pr_y_a1c0 - pr_y_a0c0)*pr_c0_a1
print("ATT", np.round(att, 6))

ATT 0.073248


In [43]:
# Box 6: Bootstrap the 95% confidence intervals (CI) for the
#        ATE/ATT estimated using the non-parametric G-Formula

def ate_nonparm_gformula(d):
    """Function to estimate the ATE using the nonparametric
    g-formula"""
    pr_c1 = np.mean(d['C'])
    pr_c0 = 1 - pr_c1

    pr_y_11 = np.mean(d.loc[(d['C'] == 1) & (d['A'] == 1), 'Y'])
    pr_y_01 = np.mean(d.loc[(d['C'] == 1) & (d['A'] == 0), 'Y'])
    pr_y_10 = np.mean(d.loc[(d['C'] == 0) & (d['A'] == 1), 'Y'])
    pr_y_00 = np.mean(d.loc[(d['C'] == 0) & (d['A'] == 0), 'Y'])
    
    return (pr_y_11 - pr_y_01)*pr_c1 + (pr_y_10 - pr_y_00)*pr_c0

## ATE ##
ate_rs = []
for i in range(200):  # Drawing 200 bootstrapped samples
    d_star = data.sample(n=data.shape[0], # Same size as input data
                         replace=True)  # Draw with replacement
    ate_rs.append(ate_nonparm_gformula(d=d_star))

print("95% Confidence limits for the ATE")
ci_perc = np.percentile(ate_rs, q=[2.5, 97.5])
print("Percentile method:    ", np.round(ci_perc, 6))
ate_se = np.std(ate_rs, ddof=1)
print("Standard error method:", np.round([ate - 1.96*ate_se,
                                          ate + 1.96*ate_se], 6))


def att_nonparm_gformula(d):
    """Function to estimate the ATT using the nonparametric
    g-formula"""
    pr_c1_a1 = np.mean(d.loc[data['A'] == 1, 'C'])
    pr_c0_a1 = 1 - pr_c1_a1

    pr_y_11 = np.mean(d.loc[(d['C'] == 1) & (d['A'] == 1), 'Y'])
    pr_y_01 = np.mean(d.loc[(d['C'] == 1) & (d['A'] == 0), 'Y'])
    pr_y_10 = np.mean(d.loc[(d['C'] == 0) & (d['A'] == 1), 'Y'])
    pr_y_00 = np.mean(d.loc[(d['C'] == 0) & (d['A'] == 0), 'Y'])
    
    return (pr_y_11 - pr_y_01)*pr_c1_a1 + (pr_y_10 - pr_y_00)*pr_c0_a1


## ATT ##
att_rs = []
for i in range(200):  # Drawing 200 bootstrapped samples
    d_star = data.sample(n=data.shape[0], # Same size as input data
                         replace=True)  # Draw with replacement
    att_rs.append(att_nonparm_gformula(d=d_star))

print("\n95% Confidence limits for the ATT")
ci_perc = np.percentile(att_rs, q=[2.5, 97.5])
print("Percentile method:    ", np.round(ci_perc, 6))
att_se = np.std(att_rs, ddof=1)
print("Standard error method:", np.round([att - 1.96*att_se,
                                          att + 1.96*att_se], 6))    

95% Confidence limits for the ATE
Percentile method:     [0.048553 0.093859]
Standard error method: [0.048678 0.098706]

95% Confidence limits for the ATT
Percentile method:     [0.050157 0.098636]
Standard error method: [0.048181 0.098315]


In [44]:
# Box 7: Non-parametric g-formula using saturated regression model (A)
data["A1"] = np.where(data['A'] == 1, 1, 0)
data["A0"] = np.where(data['A'] == 0, 1, 0)
fm = smf.ols("Y ~ A1 + A0 + A1:C + A0:C - 1", data).fit()
betas = fm.params
ate = np.mean((betas["A1"] + betas["A1:C"]*data["C"]) -
              (betas["A0"] + betas["A0:C"]*data["C"]))

print(np.round(ate, 6))

0.073692


In [ ]:
# Box 8: Non-parametric g-formula using saturated regression model (B)



### 3.2 Parametric g-formula

In [47]:
# Box 9: Parametric g-formula by hand
f = sm.families.family.Binomial()  # Using logit model unlike prev
fm_a1 = smf.glm("Y ~ C", data.loc[data["A"] == 1], family=f).fit()
fm_a0 = smf.glm("Y ~ C", data.loc[data["A"] == 0], family=f).fit()

y_a1 = fm_a1.predict(data['C'])
y_a0 = fm_a0.predict(data['C'])
ate = np.mean(y_a1 - y_a0)

print("ATE", np.round(ate, 6))

ATE 0.073692


## 4. Inverse Probability of Treatment Weighting

### 4.1 Inverse probability of treatment weighting based on the propensity score plus regression adjustment

### 4.2 Marginal structural model with stabilised weights

### 4.3 IPTW with regression adjustment

## 5. Augmented Inverse Probability Weighting

## 6. Data-Adaptive Estimation: Ensemble Learning Targeted Maximum Likelihood Estimation

## 7. Simulation

END